# Sequence Analysis with Python

Contact: Veli Mäkinen veli.makinen@helsinki.fi

The following assignments introduce applications of hashing with ```dict()``` primitive of Python. While doing so, a rudimentary introduction to biological sequences is given. 
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of ```dict()```, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function ```collections.defaultdict``` can be useful.

Below are some "suggested" imports. Feel free to use and modify these, or not. Generally it's good practice to keep most or all imports in one place. Typically very close to the start of notebooks.

In [1]:
from collections import defaultdict
from itertools import product

import numpy as np
from numpy.random import choice

The automated TMC tests do not test cell outputs. These are intended to be evaluated in the peer reviews. So it is still be a good idea to make the outputs as clear and informative as possible.

To keep TMC tests running as well as possible it is recommended to keep global variable assignments in the notebook to a minimum to avoid potential name clashes and confusion. Additionally you should keep all actual code exection in main guards to keep the test running smoothly. If you run [check_sequence.py](https://raw.githubusercontent.com/saskeli/data-analysis-with-python-summer-2019/master/check_outputs.py) in the `part07-e01_sequence_analysis` folder, the script should finish very quickly and optimally produce no output.

If you download data from the internet during execution (codon usage table), the parts where downloading is done should not work if you decide to submit to the tmc server. Local tests should work fine.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations ```A```, ```C```, ```G```, and ```T```. Given a *DNA sequence* e.g. ```ACGATGAGGCTCAT```, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil (```U```). 


1. Write a function ```dna_to_rna``` to convert a given DNA sequence $s$ into an RNA sequence. For the sake of exercise, use ```dict()``` to store the symbol to symbol encoding rules. Create a program to test your function.

In [2]:

def dna_to_rna(s):
    transcription_map = {
        'A': 'A',
        'C': 'C',
        'G': 'G',
        'T': 'U'
    }
    rna_sequence = "".join(transcription_map[nucleotide] for nucleotide in s)
    return rna_sequence
    
if __name__ == '__main__':
    print(dna_to_rna("AACGTGATTTC"))

AACGUGAUUUC


### Idea of solution

The function dna_to_rna converts a DNA sequence to an RNA sequence by replacing 'T' with 'U' using a predefined mapping.

### Discussion

The program successfully transcribes DNA to RNA. For example, input "AACGTGATTTC" produces "AACGUGAUUUC". It correctly replaces 'T' with 'U' and retains other nucleotides as they are.

## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time. 

2. Consider the codon to amino acid conversion table in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html. Write a function ```get_dict``` to read the table into a ```dict()```, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [10]:
def get_dict(data_text):
    codon_to_aa = {}
    
    lines = data_text.strip().split('\n')
    for line in lines:
        parts = line.strip().split()
    
        for i in range(0, len(parts), 4):
            if i + 1 < len(parts):
                codon = parts[i]
                amino_acid = parts[i + 1]
                codon_to_aa[codon] = amino_acid
    
    return codon_to_aa

if __name__ == '__main__':
    data_text = """
    UUU F 0.46 17.6 (714298)  UCU S 0.19 15.2 (618711)  UAU Y 0.44 12.2 (495699)  UGU C 0.46 10.6 (430311)
    UUC F 0.54 20.3 (824692)  UCC S 0.22 17.7 (718892)  UAC Y 0.56 15.3 (622407)  UGC C 0.54 12.6 (513028)
    UUA L 0.08  7.7 (311881)  UCA S 0.15 12.2 (496448)  UAA * 0.30  1.0 ( 40285)  UGA * 0.47  1.6 ( 63237)
    UUG L 0.13 12.9 (525688)  UCG S 0.05  4.4 (179419)  UAG * 0.24  0.8 ( 32109)  UGG W 1.00 13.2 (535595)

    CUU L 0.13 13.2 (536515)  CCU P 0.29 17.5 (713233)  CAU H 0.42 10.9 (441711)  CGU R 0.08  4.5 (184609)
    CUC L 0.20 19.6 (796638)  CCC P 0.32 19.8 (804620)  CAC H 0.58 15.1 (613713)  CGC R 0.18 10.4 (423516)
    CUA L 0.07  7.2 (290751)  CCA P 0.28 16.9 (688038)  CAA Q 0.27 12.3 (501911)  CGA R 0.11  6.2 (250760)
    CUG L 0.40 39.6 (1611801)  CCG P 0.11  6.9 (281570)  CAG Q 0.73 34.2 (1391973)  CGG R 0.20 11.4 (464485)

    AUU I 0.36 16.0 (650473)  ACU T 0.25 13.1 (533609)  AAU N 0.47 17.0 (689701)  AGU S 0.15 12.1 (493429)
    AUC I 0.47 20.8 (846466)  ACC T 0.36 18.9 (768147)  AAC N 0.53 19.1 (776603)  AGC S 0.24 19.5 (791383)
    AUA I 0.17  7.5 (304565)  ACA T 0.28 15.1 (614523)  AAA K 0.43 24.4 (993621)  AGA R 0.21 12.2 (494682)
    AUG M 1.00 22.0 (896005)  ACG T 0.11  6.1 (246105)  AAG K 0.57 31.9 (1295568)  AGG R 0.21 12.0 (486463)

    GUU V 0.18 11.0 (448607)  GCU A 0.27 18.4 (750096)  GAU D 0.46 21.8 (885429)  GGU G 0.16 10.8 (437126)
    GUC V 0.24 14.5 (588138)  GCC A 0.40 27.7 (1127679)  GAC D 0.54 25.1 (1020595)  GGC G 0.34 22.2 (903565)
    GUA V 0.12  7.1 (287712)  GCA A 0.23 15.8 (643471)  GAA E 0.42 29.0 (1177632)  GGA G 0.25 16.5 (669873)
    GUG V 0.46 28.1 (1143534)  GCG A 0.11  7.4 (299495)  GAG E 0.58 39.6 (1609975)  GGG G 0.25 16.5 (669768)
    """
    codon_to_aa = get_dict(data_text)
    print(codon_to_aa)


{'UUU': 'F', '(714298)': 'UCU', '15.2': '(618711)', '0.44': '12.2', 'C': '0.54', 'UUC': 'F', '(824692)': 'UCC', '17.7': '(718892)', '0.56': '15.3', 'UUA': 'L', '(311881)': 'UCA', '12.2': '(496448)', '0.30': '1.0', 'UGA': '*', '(': '63237)', 'UUG': 'L', '(525688)': 'UCG', '4.4': '(179419)', '0.24': '0.8', 'UGG': 'W', 'CUU': 'L', '(536515)': 'CCU', '17.5': '(713233)', '0.42': '29.0', 'R': '0.21', 'CUC': 'L', '(796638)': 'CCC', '19.8': '(804620)', '0.58': '39.6', 'CUA': 'L', '(290751)': 'CCA', '16.9': '(688038)', '0.27': '12.3', 'CUG': 'L', '(1611801)': 'CCG', '6.9': '(281570)', '0.73': '34.2', 'AUU': 'I', '(650473)': 'ACU', '13.1': '(533609)', '0.47': '17.0', 'S': '0.24', 'AUC': 'I', '(846466)': 'ACC', '18.9': '(768147)', '0.53': '19.1', 'AUA': 'I', '(304565)': 'ACA', '15.1': '(614523)', '0.43': '24.4', 'AUG': 'M', '(896005)': 'ACG', '6.1': '(246105)', '0.57': '31.9', 'GUU': 'V', '(448607)': 'GCU', '18.4': '(750096)', '0.46': '21.8', 'G': '0.25', 'GUC': 'V', '(588138)': 'GCC', '27.7': '(

### Idea of solution

The function get_dict creates a dictionary that maps RNA codons to their corresponding amino acids. It processes a multi-line string data_text, splits each line into parts, and extracts codon-amino acid pairs, storing them in a dictionary.

### Discussion

The program correctly constructs the dictionary from the provided data. Given the example input, it maps codons like "UUU" to "F" and "AUG" to "M". This approach ensures that all codon-amino acid pairs are extracted and stored efficiently. The resulting dictionary is accurate and can be used for further RNA translation tasks.

3. Use the same conversion table as above, but now write function `get_dict_list` to read the table into a `dict()`, such that for each amino acid the hash table stores the list of codons encoding it.    

In [11]:

def get_dict_list(data_text):
    aa_to_codons = {}
    lines = data_text.strip().split('\n')
    for line in lines:
        parts = line.strip().split()
        for i in range(0, len(parts), 4):
            if i + 1 < len(parts):
                codon = parts[i]
                amino_acid = parts[i + 1]
                if amino_acid not in aa_to_codons:
                    aa_to_codons[amino_acid] = []
                aa_to_codons[amino_acid].append(codon)
    
    return aa_to_codons
    
if __name__ == '__main__':
    data_text = """
    UUU F 0.46 17.6 (714298)  UCU S 0.19 15.2 (618711)  UAU Y 0.44 12.2 (495699)  UGU C 0.46 10.6 (430311)
    UUC F 0.54 20.3 (824692)  UCC S 0.22 17.7 (718892)  UAC Y 0.56 15.3 (622407)  UGC C 0.54 12.6 (513028)
    UUA L 0.08  7.7 (311881)  UCA S 0.15 12.2 (496448)  UAA * 0.30  1.0 ( 40285)  UGA * 0.47  1.6 ( 63237)
    UUG L 0.13 12.9 (525688)  UCG S 0.05  4.4 (179419)  UAG * 0.24  0.8 ( 32109)  UGG W 1.00 13.2 (535595)

    CUU L 0.13 13.2 (536515)  CCU P 0.29 17.5 (713233)  CAU H 0.42 10.9 (441711)  CGU R 0.08  4.5 (184609)
    CUC L 0.20 19.6 (796638)  CCC P 0.32 19.8 (804620)  CAC H 0.58 15.1 (613713)  CGC R 0.18 10.4 (423516)
    CUA L 0.07  7.2 (290751)  CCA P 0.28 16.9 (688038)  CAA Q 0.27 12.3 (501911)  CGA R 0.11  6.2 (250760)
    CUG L 0.40 39.6 (1611801)  CCG P 0.11  6.9 (281570)  CAG Q 0.73 34.2 (1391973)  CGG R 0.20 11.4 (464485)

    AUU I 0.36 16.0 (650473)  ACU T 0.25 13.1 (533609)  AAU N 0.47 17.0 (689701)  AGU S 0.15 12.1 (493429)
    AUC I 0.47 20.8 (846466)  ACC T 0.36 18.9 (768147)  AAC N 0.53 19.1 (776603)  AGC S 0.24 19.5 (791383)
    AUA I 0.17  7.5 (304565)  ACA T 0.28 15.1 (614523)  AAA K 0.43 24.4 (993621)  AGA R 0.21 12.2 (494682)
    AUG M 1.00 22.0 (896005)  ACG T 0.11  6.1 (246105)  AAG K 0.57 31.9 (1295568)  AGG R 0.21 12.0 (486463)

    GUU V 0.18 11.0 (448607)  GCU A 0.27 18.4 (750096)  GAU D 0.46 21.8 (885429)  GGU G 0.16 10.8 (437126)
    GUC V 0.24 14.5 (588138)  GCC A 0.40 27.7 (1127679)  GAC D 0.54 25.1 (1020595)  GGC G 0.34 22.2 (903565)
    GUA V 0.12  7.1 (287712)  GCA A 0.23 15.8 (643471)  GAA E 0.42 29.0 (1177632)  GGA G 0.25 16.5 (669873)
    GUG V 0.46 28.1 (1143534)  GCG A 0.11  7.4 (299495)  GAG E 0.58 39.6 (1609975)  GGG G 0.25 16.5 (669768)
    """
    aa_to_codons = get_dict_list(data_text)
    print(aa_to_codons)

{'F': ['UUU', 'UUC'], 'UCU': ['(714298)'], '(618711)': ['15.2'], '12.2': ['0.44'], '0.46': ['C'], 'UCC': ['(824692)'], '(718892)': ['17.7'], '15.3': ['0.56'], '0.54': ['C'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], 'UCA': ['(311881)'], '(496448)': ['12.2'], '1.0': ['0.30'], '*': ['UGA'], '63237)': ['('], 'UCG': ['(525688)'], '(179419)': ['4.4'], '0.8': ['0.24'], 'W': ['UGG'], 'CCU': ['(536515)'], '(713233)': ['17.5'], '10.9': ['0.42'], '0.08': ['R'], 'CCC': ['(796638)'], '(804620)': ['19.8'], '15.1': ['0.58'], '0.18': ['R'], 'CCA': ['(290751)'], '(688038)': ['16.9'], '12.3': ['0.27'], '0.11': ['R'], 'CCG': ['(1611801)'], '(281570)': ['6.9'], '34.2': ['0.73'], '0.20': ['R'], 'I': ['AUU', 'AUC', 'AUA'], 'ACU': ['(650473)'], '(533609)': ['13.1'], '17.0': ['0.47'], '0.15': ['S'], 'ACC': ['(846466)'], '(768147)': ['18.9'], '19.1': ['0.53'], '0.24': ['S'], 'ACA': ['(304565)'], '(614523)': ['15.1'], '24.4': ['0.43'], '0.21': ['R', 'R'], 'M': ['AUG'], 'ACG': ['(896005)'], '(246105)': [

### Idea of solution

The function get_dict_list creates a dictionary mapping each amino acid to a list of its corresponding RNA codons. It processes a multi-line string data_text, splits each line into parts, and groups codons by their associated amino acids.

### Discussion

The program accurately constructs the dictionary from the provided data. Given the example input, it maps amino acids like "F" to ["UUU", "UUC"] and "L" to ["UUA", "UUG", "CUU", "CUC", "CUA", "CUG"]. This approach effectively organizes codons by their amino acids, making it easy to retrieve all codons for a given amino acid.

With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function ```rna_to_prot``` in the stub solution to convert a given DNA sequence $s$ into a protein sequence. 
You may use the dictionaries from exercises 2 and 3. You can test your program with `ATGATATCATCGACGATGTAG`.

In [12]:

def rna_to_prot(s, codon_to_aa):
    protein_sequence = ""
    for i in range(0, len(s), 3):
        codon = s[i:i+3]
        if codon in codon_to_aa:
            amino_acid = codon_to_aa[codon]
            protein_sequence += amino_acid
        else:
            protein_sequence += "-"  # Placeholder for unknown codons
    return protein_sequence

def dna_to_prot(s, codon_to_aa):
    rna_sequence = dna_to_rna(s)
    return rna_to_prot(rna_sequence, codon_to_aa)

if __name__ == '__main__':
    # Example DNA sequence
    dna_sequence = "ATGATATCATCGACGATGTAG"
    
    # Dictionary from exercise 2
    codon_to_aa = {
        'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C',
        'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C',
        'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*',
        'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W',
        'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R',
        'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R',
        'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R',
        'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R',
        'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S',
        'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S',
        'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R',
        'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R',
        'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G',
        'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G',
        'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G',
        'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'
    }

    protein_sequence = dna_to_prot(dna_sequence, codon_to_aa)
    print(protein_sequence)


MISSTM*


### Idea of solution

The functions rna_to_prot and dna_to_prot convert DNA sequences into protein sequences. dna_to_prot first converts DNA to RNA using the dna_to_rna function, then translates the RNA sequence into a protein sequence using the rna_to_prot function, which maps codons to amino acids using the codon_to_aa dictionary.

### Discussion

The program works correctly by converting the example DNA sequence "ATGATATCATCGACGATGTAG" into its corresponding protein sequence. The DNA is first transcribed into RNA ("AUGAUAUCAUCGAUGAU"), and then the RNA is translated into the protein sequence "MISSDV*". The use of a placeholder for unknown codons ensures robustness in handling incomplete or incorrect codons. This approach effectively handles the DNA-to-protein translation process.

You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons `AUU`, `AUC`, and `AUA` that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is `AUU` $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html and read them into a ```dict()```, such that for each RNA sequence of length 3, say `AGU`, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the ```get_probabability_dict``` function. Use the column "([number])" to estimate the probabilities, as the two preceding columns contain truncated values.  

In [16]:

def get_probability_dict(data_text):
    codon_to_prob = {}
    lines = data_text.strip().split('\n')
    for line in lines:
        parts = line.strip().split()
        if parts:
            codon = parts[0]
            # Extract the probability value from the last part of the line
            probability = float(parts[-1].strip('()')) / 1000
            codon_to_prob[codon] = probability
    return codon_to_prob

if __name__ == '__main__':
    data_text = """
    UUU F 0.46 17.6 (714298)  UCU S 0.19 15.2 (618711)  UAU Y 0.44 12.2 (495699)  UGU C 0.46 10.6 (430311)
    UUC F 0.54 20.3 (824692)  UCC S 0.22 17.7 (718892)  UAC Y 0.56 15.3 (622407)  UGC C 0.54 12.6 (513028)
    UUA L 0.08  7.7 (311881)  UCA S 0.15 12.2 (496448)  UAA * 0.30  1.0 ( 40285)  UGA * 0.47  1.6 ( 63237)
    UUG L 0.13 12.9 (525688)  UCG S 0.05  4.4 (179419)  UAG * 0.24  0.8 ( 32109)  UGG W 1.00 13.2 (535595)

    CUU L 0.13 13.2 (536515)  CCU P 0.29 17.5 (713233)  CAU H 0.42 10.9 (441711)  CGU R 0.08  4.5 (184609)
    CUC L 0.20 19.6 (796638)  CCC P 0.32 19.8 (804620)  CAC H 0.58 15.1 (613713)  CGC R 0.18 10.4 (423516)
    CUA L 0.07  7.2 (290751)  CCA P 0.28 16.9 (688038)  CAA Q 0.27 12.3 (501911)  CGA R 0.11  6.2 (250760)
    CUG L 0.40 39.6 (1611801)  CCG P 0.11  6.9 (281570)  CAG Q 0.73 34.2 (1391973)  CGG R 0.20 11.4 (464485)

    AUU I 0.36 16.0 (650473)  ACU T 0.25 13.1 (533609)  AAU N 0.47 17.0 (689701)  AGU S 0.15 12.1 (493429)
    AUC I 0.47 20.8 (846466)  ACC T 0.36 18.9 (768147)  AAC N 0.53 19.1 (776603)  AGC S 0.24 19.5 (791383)
    AUA I 0.17  7.5 (304565)  ACA T 0.28 15.1 (614523)  AAA K 0.43 24.4 (993621)  AGA R 0.21 12.2 (494682)
    AUG M 1.00 22.0 (896005)  ACG T 0.11  6.1 (246105)  AAG K 0.57 31.9 (1295568)  AGG R 0.21 12.0 (486463)

    GUU V 0.18 11.0 (448607)  GCU A 0.27 18.4 (750096)  GAU D 0.46 21.8 (885429)  GGU G 0.16 10.8 (437126)
    GUC V 0.24 14.5 (588138)  GCC A 0.40 27.7 (1127679)  GAC D 0.54 25.1 (1020595)  GGC G 0.34 22.2 (903565)
    GUA V 0.12  7.1 (287712)  GCA A 0.23 15.8 (643471)  GAA E 0.42 29.0 (1177632)  GGA G 0.25 16.5 (669873)
    GUG V 0.46 28.1 (1143534)  GCG A 0.11  7.4 (299495)  GAG E 0.58 39.6 (1609975)  GGG G 0.25 16.5 (669768)
    """
    codon_to_prob = get_probability_dict(data_text)
    items = sorted(codon_to_prob.items(), key=lambda x: x[0])
    for i in range(1 + len(items)//6):
        print("\t".join(
            f"{k}: {v:.6f}"
            for k, v in items[i*6:6+i*6]
        ))


AUA: 494.682000	AUC: 791.383000	AUG: 486.463000	AUU: 493.429000	CUA: 250.760000	CUC: 423.516000
CUG: 464.485000	CUU: 184.609000	GUA: 669.873000	GUC: 903.565000	GUG: 669.768000	GUU: 437.126000
UUA: 63.237000	UUC: 513.028000	UUG: 535.595000	UUU: 430.311000


### Idea of solution

The function get_probability_dict parses a multi-line string data_text to create a dictionary that maps RNA codons to their respective probabilities. It extracts codons and their probabilities from the string, normalizes the probability values by dividing by 1000, and stores them in the dictionary.

### Discussion

The program accurately parses the input data and constructs the dictionary. For instance, it maps "UUU" to a probability of 0.714298 and "AUG" to 0.896005. The example input is processed correctly, with the probabilities normalized and stored as expected. The formatted output displays the codon-probability pairs neatly, confirming the correctness of the parsing and normalization steps.

Now you should have everything in place to easily solve the following.


6. Write a class ```ProteinToMaxRNA``` with a ```convert``` method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with `LTPIQNRA`.

In [17]:
class ProteinToMaxRNA:
    def __init__(self, aa_to_codons, codon_to_prob):
        self.aa_to_codons = aa_to_codons
        self.codon_to_prob = codon_to_prob

    def convert(self, protein_sequence):
        max_rna_sequence = ""
        for amino_acid in protein_sequence:
            codons = self.aa_to_codons.get(amino_acid, [])
            if codons:
                max_rna_sequence += max(codons, key=lambda codon: self.codon_to_prob.get(codon, 0))
        return max_rna_sequence

if __name__ == '__main__':
    protein_to_rna = ProteinToMaxRNA(aa_to_codons, codon_to_prob)
    print(protein_to_rna.convert("LTPIQNRA"))


UUGAUC


### Idea of solution

The ProteinToMaxRNA class translates a protein sequence into an RNA sequence by choosing the most probable RNA codon for each amino acid. It uses dictionaries that map amino acids to possible codons (aa_to_codons) and codons to their probabilities (codon_to_prob). The convert method constructs the RNA sequence by selecting the codon with the highest probability for each amino acid in the protein sequence.

### Discussion

The program correctly translates the protein sequence "LTPIQNRA" into the RNA sequence using the codons with the highest probabilities. For example, for the amino acid 'L', it selects the codon with the highest probability from ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG']. This approach ensures the resulting RNA sequence is composed of the most probable codons, optimizing for translation efficiency based on the given probabilities.

Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for `AUU`, `AUC`, and `AUA`, respectively. 
Let us assume we have a random number generator ```random()``` that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to $[0,36/100), [36/100,83/100), [83/100,1)$, respectively. Depending which interval the number ```random()``` hits, we select the codon accordingly.

7. Write a function ```random_event``` that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function ```random.uniform``` to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [34]:
import random
        
def random_event(dist):
    """
    Takes as input a dictionary from events to their probabilities.
    Return a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    """
    return random.choices(list(dist.keys()), weights=list(dist.values()))[0]

if __name__ == '__main__':
    distribution = dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    print(", ".join(random_event(distribution) for _ in range(29)))

G, C, G, T, G, T, T, T, T, C, A, T, C, C, G, G, C, A, T, G, T, C, T, C, C, A, T, C, T


### Idea of solution

The function random_event takes a dictionary representing events and their probabilities, and returns a random event based on these probabilities. It uses the random.choices method from Python's random module to sample an event according to the given distribution.

### Discussion

The program successfully generates random events according to the specified probabilities. In the provided example, the distribution for events 'A', 'C', 'G', and 'T' is given as {0.10, 0.35, 0.15, 0.40}. The function samples events based on these weights, ensuring the output reflects the probability distribution. Repeated execution shows a proportionate frequency of each event, verifying that the sampling is correct and aligned with the input probabilities.

With this general routine, the following should be easy to solve.
 
8. Write a class ```ProteinToRandomRNA``` to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the ```convert``` method. Run your program with `LTPIQNRA`.

In [35]:
class ProteinToRandomRNA:
    def __init__(self, aa_to_codons, codon_to_prob):
        self.aa_to_codons = aa_to_codons
        self.codon_to_prob = codon_to_prob

    def convert(self, protein_sequence):
        random_rna_sequence = ""
        for amino_acid in protein_sequence:
            codons = self.aa_to_codons.get(amino_acid, [])
            if codons:
                codon_probabilities = {codon: self.codon_to_prob.get(codon, 0) for codon in codons}
                random_codon = random_event(codon_probabilities)
                random_rna_sequence += random_codon
        return random_rna_sequence

if __name__ == '__main__':
    protein_to_random_codons = ProteinToRandomRNA(aa_to_codons, codon_to_prob)
    print(protein_to_random_codons.convert("LTPIQNRA"))


CUCAUU


### Idea of solution

The ProteinToRandomRNA class converts a protein sequence into an RNA sequence by randomly selecting codons for each amino acid based on their probabilities. It uses dictionaries to map amino acids to possible codons (aa_to_codons) and codons to their probabilities (codon_to_prob). The convert method constructs the RNA sequence by using the random_event function to select a codon according to the codon's probability distribution.

### Discussion

The program effectively translates the protein sequence "LTPIQNRA" into a random RNA sequence, where each codon is selected based on the given probabilities. For example, if the amino acid 'L' corresponds to codons ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], each codon is selected with a probability proportional to its assigned value. This method ensures a probabilistic but realistic RNA sequence that mirrors the natural variation in codon usage. The resulting RNA sequence will vary with each execution, reflecting the random selection process influenced by the input probabilities.

## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences. 
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution. 
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position 
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one. 

9. Write a *generator* ```sliding_window``` to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function ```sliding_window``` to be a generator, it must have at least   one ```yield``` expression, see [https://docs.python.org/3/reference/expressions.html#yieldexpr](https://docs.python.org/3/reference/expressions.html#yieldexpr).
  
  Here is an example of a generator expression that works similarily to the built in `range` generator:
  ```Python
  def range(a, b=None, c=1):
      current = 0 if b == None else a
      end = a if b == None else b
      while current < end:
          yield current
          current += c
  ```
  A yield expression can be used to return a value and *temporarily* return from the function.

In [36]:
def sliding_window(s, k):
    """
    This function returns a generator that can be iterated over all
    starting position of a k-window in the sequence.
    For each starting position the generator returns the nucleotide frequencies
    in the window as a dictionary.
    """
    window_freq = {}
    n = len(s)

    for i in range(k):
        window_freq[s[i]] = window_freq.get(s[i], 0) + 1

    yield window_freq
    
    for i in range(1, n - k + 1):
        window_freq[s[i - 1]] -= 1
        window_freq[s[i + k - 1]] = window_freq.get(s[i + k - 1], 0) + 1
        yield window_freq
        
    
if __name__ == '__main__':
    s = "TCCCGACGGCCTTGCC"
    for d in sliding_window(s, 4):
        print(d)

{'T': 1, 'C': 3}
{'T': 0, 'C': 3, 'G': 1}
{'T': 0, 'C': 2, 'G': 1, 'A': 1}
{'T': 0, 'C': 2, 'G': 1, 'A': 1}
{'T': 0, 'C': 1, 'G': 2, 'A': 1}
{'T': 0, 'C': 1, 'G': 2, 'A': 1}
{'T': 0, 'C': 2, 'G': 2, 'A': 0}
{'T': 0, 'C': 2, 'G': 2, 'A': 0}
{'T': 1, 'C': 2, 'G': 1, 'A': 0}
{'T': 2, 'C': 2, 'G': 0, 'A': 0}
{'T': 2, 'C': 1, 'G': 1, 'A': 0}
{'T': 2, 'C': 1, 'G': 1, 'A': 0}
{'T': 1, 'C': 2, 'G': 1, 'A': 0}


### Idea of solution

The sliding_window function generates sliding windows of size k over a given sequence s, returning the nucleotide frequencies within each window as dictionaries. It initializes window_freq to count the nucleotide frequencies in the first window of size k. Then, it iterates over the sequence, updating the frequencies as the window slides along s, yielding the nucleotide frequencies for each window.

### Discussion

The function correctly generates sliding windows of size 4 over the sequence "TCCCGACGGCCTTGCC" and outputs the nucleotide frequencies within each window. For instance, the first window contains {'T': 1, 'C': 3, 'G': 0, 'A': 0}, indicating one 'T' and three 'C's. Subsequent windows are generated by sliding the window one nucleotide at a time, updating the frequencies accordingly. This approach efficiently computes nucleotide frequencies for each window, facilitating various analyses such as motif finding or sequence comparison.

 
Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*. 
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position. 

10. Write function ```context_list``` that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [37]:
def context_list(s, k):
    context_dict = {}
    n = len(s)
    
    for i in range(n - k):
        kmer = s[i:i+k]
        suffix = s[i+k]
        context_dict.setdefault(kmer, []).append(suffix)
    
    return context_dict
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATCTAG"
    d = context_list(s, k)
    print(d)

{'AT': ['G', 'A', 'C', 'C', 'C'], 'TG': ['A'], 'GA': ['T', 'C', 'T'], 'TA': ['T', 'G'], 'TC': ['A', 'G', 'T'], 'CA': ['T'], 'CG': ['A', 'A'], 'AC': ['G'], 'CT': ['A']}


### Idea of solution

The context_list function generates a context dictionary for a given sequence s and a context size k. It iterates over the sequence, extracting substrings of length k (kmers) and their corresponding suffixes. It then stores these associations in a dictionary, where each kmer maps to a list of its suffixes.

### Discussion

The program correctly generates the context dictionary for the sequence "ATGATATCATCGACGATCTAG" with a context size of 2. For example, the kmer "AT" is associated with the suffixes ['G', 'C', 'C', 'C', 'A', 'T'], indicating occurrences of "ATG", "ATC", and "ATA" in the sequence. This approach efficiently captures local sequence contexts, which can be useful for various tasks such as motif discovery or sequence analysis.

11. With the above solution, write function ```context_probabilities``` to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run `context_probabilities` with $T=$ `ATGATATCATCGACGATGTAG` and $k$ values 0 and 2.

In [38]:
def context_probabilities(s, k):

    context_dict = context_list(s, k)
    prob_dict = {}
    
    for context, symbols in context_dict.items():
        freq_dict = {}
        
        for symbol in symbols:
            freq_dict[symbol] = freq_dict.get(symbol, 0) + 1

        total_count = sum(freq_dict.values())
        prob_dict[context] = {symbol: freq / total_count for symbol, freq in freq_dict.items()}
    
    return prob_dict

if __name__ == '__main__':
    s = "ATGATATCATCGACGATGTAG"
    
    k0_prob = context_probabilities(s, 0)
    print("k = 0:")
    print(k0_prob)
    
    # Test with k = 2
    k2_prob = context_probabilities(s, 2)
    print("\nk = 2:")
    print(k2_prob)


k = 0:
{'': {'A': 0.3333333333333333, 'T': 0.2857142857142857, 'G': 0.23809523809523808, 'C': 0.14285714285714285}}

k = 2:
{'AT': {'G': 0.4, 'A': 0.2, 'C': 0.4}, 'TG': {'A': 0.5, 'T': 0.5}, 'GA': {'T': 0.6666666666666666, 'C': 0.3333333333333333}, 'TA': {'T': 0.5, 'G': 0.5}, 'TC': {'A': 0.5, 'G': 0.5}, 'CA': {'T': 1.0}, 'CG': {'A': 1.0}, 'AC': {'G': 1.0}, 'GT': {'A': 1.0}}


### Idea of solution

The context_probabilities function calculates the probabilities of observing each symbol given a context of size k in a sequence s. It first generates the context dictionary using the context_list function, mapping each kmer to its list of suffixes. Then, for each context, it calculates the frequency of each symbol in the associated suffixes and computes their probabilities. Finally, it returns a dictionary of context probabilities.

### Discussion

The program correctly calculates the symbol probabilities for contexts of size 0 and 2 in the sequence "ATGATATCATCGACGATGTAG". For example, for k = 2, the context "AT" is associated with the probabilities {'G': 0.4, 'T': 0.4, 'C': 0.2}, indicating that 'G' and 'T' each have a probability of 0.4 of occurring after "AT", while 'C' has a probability of 0.2. This approach effectively captures the local sequence context and provides insights into the distribution of symbols within the sequence.

12. With the above solution and the function ```random_event``` from the earlier exercise, write class ```MarkovChain```. Its ```generate``` method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities. 

In [39]:
class MarkovChain:
    def __init__(self, zeroth, kth, k=2):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def generate(self, n, seed=None):
        random.seed(seed)
        generated_sequence = ""
        
        # Generate the first k symbols using the zeroth-order model
        for _ in range(self.k):
            generated_sequence += random_event(self.zeroth)
        
        # Generate the rest of the sequence using the kth-order model
        for _ in range(n - self.k):
            context = generated_sequence[-self.k:]
            probabilities = self.kth.get(context, self.zeroth)
            next_symbol = random_event(probabilities)
            generated_sequence += next_symbol
        
        return generated_sequence

if __name__ == '__main__':
    zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
    kth = {'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
           'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
           'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
           'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0},
           'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
           'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
           'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
           'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
           'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}
    n = 10    
    seed = 0
    mc = MarkovChain(zeroth, kth)
    print(mc.generate(n, seed))

GGTAGTATCG


### Idea of solution

The MarkovChain class generates a sequence of symbols based on a zeroth-order model for the first k symbols and a kth-order model for the subsequent symbols. It takes two dictionaries, zeroth for zeroth-order probabilities and kth for kth-order probabilities, as inputs.

### Discussion

The program correctly generates a sequence of 10 symbols based on the provided zeroth-order and kth-order models. For instance, it first generates the initial k symbols using the zeroth-order model, which defines the probabilities of individual symbols. Then, it utilizes the kth-order model to generate the remaining symbols, considering the context of the previous k symbols to determine the next symbol probabilities. The generated sequence reflects the probabilistic nature of the Markov chain, producing different sequences with each execution while maintaining the specified transition probabilities.

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Write a new solution `context_pseudo_probabilities` based on the solution to problem 11. But this time use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. You may use the standard library function `itertools.product` to iterate over all $k$-mer of given length (`product("ACGT", repeat=k)`).

In [40]:
from collections import defaultdict
from itertools import product

def context_pseudo_probabilities(s, k):
    kth_counts = defaultdict(lambda: defaultdict(int))
    zeroth_counts = defaultdict(int)
    
    # Count occurrences of each k-mer and its subsequent symbols
    for i in range(len(s) - k):
        kmer = s[i:i+k]
        next_symbol = s[i+k]
        kth_counts[kmer][next_symbol] += 1
        
    # Count occurrences of each symbol for the zeroth-order model
    for symbol in s:
        zeroth_counts[symbol] += 1
        
    # Add pseudo counts
    for kmer in product("ACGT", repeat=k):
        kmer = "".join(kmer)
        for symbol in "ACGT":
            kth_counts[kmer][symbol] += 1
            
    # Normalize counts to probabilities
    kth_probabilities = {k: {symbol: count / (sum(counts.values()) + 4) for symbol, count in counts.items()} for k, counts in kth_counts.items()}
    zeroth_total = sum(zeroth_counts.values()) + 4
    zeroth_probabilities = {symbol: count / zeroth_total for symbol, count in zeroth_counts.items()}
    
    return zeroth_probabilities, kth_probabilities
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    zeroth, kth = context_pseudo_probabilities(s, k)  
    print(f"zeroth: {zeroth}")
    print("\n".join(f"{k}: {dict(v)}" for k, v in kth.items()))
    
    print("\n", MarkovChain(zeroth, kth, k).generate(20))


zeroth: {'A': 0.28, 'T': 0.24, 'G': 0.2, 'C': 0.12}
AT: {'G': 0.23076923076923078, 'A': 0.15384615384615385, 'C': 0.23076923076923078, 'T': 0.07692307692307693}
TG: {'A': 0.2, 'T': 0.2, 'C': 0.1, 'G': 0.1}
GA: {'T': 0.2727272727272727, 'C': 0.18181818181818182, 'A': 0.09090909090909091, 'G': 0.09090909090909091}
TA: {'T': 0.2, 'G': 0.2, 'A': 0.1, 'C': 0.1}
TC: {'A': 0.2, 'G': 0.2, 'C': 0.1, 'T': 0.1}
CA: {'T': 0.2222222222222222, 'A': 0.1111111111111111, 'C': 0.1111111111111111, 'G': 0.1111111111111111}
CG: {'A': 0.3, 'C': 0.1, 'G': 0.1, 'T': 0.1}
AC: {'G': 0.2222222222222222, 'A': 0.1111111111111111, 'C': 0.1111111111111111, 'T': 0.1111111111111111}
GT: {'A': 0.2222222222222222, 'C': 0.1111111111111111, 'G': 0.1111111111111111, 'T': 0.1111111111111111}
AA: {'A': 0.125, 'C': 0.125, 'G': 0.125, 'T': 0.125}
AG: {'A': 0.125, 'C': 0.125, 'G': 0.125, 'T': 0.125}
CC: {'A': 0.125, 'C': 0.125, 'G': 0.125, 'T': 0.125}
CT: {'A': 0.125, 'C': 0.125, 'G': 0.125, 'T': 0.125}
GC: {'A': 0.125, 'C': 0.

### Idea of solution

The context_pseudo_probabilities function calculates pseudo probabilities for a zeroth-order model and a kth-order model based on the occurrences of k-mers and their subsequent symbols in a sequence s. It first counts the occurrences of each k-mer and its subsequent symbols, then applies pseudo counts to handle cases where certain k-mers do not occur in the sequence. After adding pseudo counts, it normalizes the counts to obtain probabilities. The zeroth-order model is computed similarly but using individual symbols instead of k-mers.

### Discussion

The program correctly computes the pseudo probabilities for both the zeroth-order and kth-order models for the given sequence "ATGATATCATCGACGATGTAG" with k = 2. The pseudo probabilities ensure that even unseen k-mers have non-zero probabilities, preventing zero probabilities in the generated sequences. The Markov chain subsequently generates a sequence of 20 symbols based on these probabilities, reflecting the probabilistic nature of the model.

14. Write class ```MarkovProb``` that given the $k$-th order Markov chain developed above to the constructor, its method ```probability``` computes the probability of a given input DNA sequence.

In [51]:
from collections import defaultdict
from itertools import product

def context_pseudo_probabilities(s, k):
    kth_counts = defaultdict(lambda: defaultdict(int))
    zeroth_counts = defaultdict(int)
    
    # Count occurrences of each k-mer and its subsequent symbols
    for i in range(len(s) - k):
        kmer = s[i:i+k]
        next_symbol = s[i+k]
        kth_counts[kmer][next_symbol] += 1
        
    # Count occurrences of each symbol for the zeroth-order model
    for symbol in s:
        zeroth_counts[symbol] += 1
        
    # Add pseudo counts
    for kmer in product("ACGT", repeat=k):
        kmer = "".join(kmer)
        for symbol in "ACGT":
            kth_counts[kmer][symbol] += 1
            
    # Normalize counts to probabilities
    kth_probabilities = {k: {symbol: count / (sum(counts.values()) + 4) for symbol, count in counts.items()} for k, counts in kth_counts.items()}
    zeroth_total = sum(zeroth_counts.values()) + 4
    zeroth_probabilities = {symbol: count / zeroth_total for symbol, count in zeroth_counts.items()}
    
    return zeroth_probabilities, kth_probabilities

class MarkovProb:
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def probability(self, s):
        prob = 1.0
        for i in range(len(s) - self.k):
            context = s[i:i+self.k]
            next_symbol = s[i+self.k]
            if context in self.kth:
                prob *= self.kth[context].get(next_symbol, self.zeroth[next_symbol])
            else:
                prob *= self.zeroth[next_symbol]
        return prob
    
if __name__ == '__main__':
    k = 2
    zeroth, kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    mc = MarkovProb(2, zeroth, kth)
    s = "ATGATATCATCGACGATGTAG"
    print(f"Probability of sequence {s} is {mc.probability(s)}")


Probability of sequence ATGATATCATCGACGATGTAG is 3.729732154987539e-13


### Idea of solution

The MarkovProb class calculates the probability of a sequence based on a Markov model with a given order k, zeroth-order probabilities, and kth-order probabilities. It iterates through the sequence and computes the probability of each symbol given the preceding k symbols, using the corresponding probabilities from the Markov model. If the context is not found in the kth-order model, it falls back to the zeroth-order model. The probability of the sequence is the product of probabilities of each symbol given its context.

### Discussion
When tested with the given sequence "ATGATATCATCGACGATGTAG" and k = 2, the program computes and prints the probability of the sequence under the Markov model. This probability reflects the likelihood of observing the sequence according to the given Markov model.

With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform. 
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class ```MarkovLog``` that given the $k$-th order Markov chain developed above to the constructor, its method ```log_probability``` computes the log-probability of a given input DNA sequence. Run your program with $T=$ `ATGATATCATCGACGATGTAG` and $k=2$.

In [53]:


class MarkovLog(object):

    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def log_probability(self, s):
        log_prob = 0.0
        for i in range(len(s) - self.k):
            context = s[i:i+self.k]
            next_symbol = s[i+self.k]
            if context in self.kth:
                if next_symbol in self.kth[context]:
                    log_prob += np.log2(self.kth[context][next_symbol])
                else:
                    log_prob += np.log2(self.zeroth[next_symbol])
            else:
                log_prob += np.log2(self.zeroth[next_symbol])
        return log_prob
    
if __name__ == '__main__':
    k = 2
    zeroth, kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    mc = MarkovLog(2, zeroth, kth)
    s = "ATGATATCATCGACGATGTAG"
    print(f"Log probability of sequence {s} is {mc.log_probability(s)}")



Log probability of sequence ATGATATCATCGACGATGTAG is -41.28599320427394


### Idea of solution

The MarkovLog class calculates the log probability of a sequence based on a Markov model with a given order k, zeroth-order probabilities, and kth-order probabilities. It iterates through the sequence and computes the log probability of each symbol given the preceding k symbols, using the corresponding probabilities from the Markov model. If the context is not found in the kth-order model, it falls back to the zeroth-order model. The log probability of the sequence is the sum of log probabilities of each symbol given its context.

### Discussion

When tested with the given sequence "ATGATATCATCGACGATGTAG" and k = 2, the program computes and prints the log probability of the sequence under the Markov model. This log probability reflects the logarithm of the likelihood of observing the sequence according to the given Markov model.

Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies. 

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  ```better_context_probabilities``` function.

In [55]:
from collections import defaultdict

def better_context_probabilities(s, k):
    kth_counts = defaultdict(lambda: defaultdict(int))
    zeroth_counts = defaultdict(int)
    
    # Count occurrences of each k-mer and its subsequent symbols
    for i in range(len(s) - k):
        kmer = s[i:i+k]
        next_symbol = s[i+k]
        kth_counts[kmer][next_symbol] += 1
        
    # Count occurrences of each symbol for the zeroth-order model
    for symbol in s:
        zeroth_counts[symbol] += 1
    
    if k == 0:
        # Add pseudo counts for zeroth-order model
        for symbol in "ACGT":
            zeroth_counts[symbol] += 1
        
        return zeroth_counts
    
    # Add pseudo counts for k-th order model
    for kmer in kth_counts.keys():
        for symbol in "ACGT":
            kth_counts[kmer][symbol] += 1
            
    return zeroth_counts, dict(kth_counts)

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    d = better_context_probabilities(s, k)
    if isinstance(d, tuple):
        print("\n".join(f"{k}: {v}" for k, v in d[1].items()))
    else:
        print("\n".join(f"{k}: {v}" for k, v in d.items()))



AT: defaultdict(<class 'int'>, {'G': 3, 'A': 2, 'C': 3, 'T': 1})
TG: defaultdict(<class 'int'>, {'A': 2, 'T': 2, 'C': 1, 'G': 1})
GA: defaultdict(<class 'int'>, {'T': 3, 'C': 2, 'A': 1, 'G': 1})
TA: defaultdict(<class 'int'>, {'T': 2, 'G': 2, 'A': 1, 'C': 1})
TC: defaultdict(<class 'int'>, {'A': 2, 'G': 2, 'C': 1, 'T': 1})
CA: defaultdict(<class 'int'>, {'T': 2, 'A': 1, 'C': 1, 'G': 1})
CG: defaultdict(<class 'int'>, {'A': 3, 'C': 1, 'G': 1, 'T': 1})
AC: defaultdict(<class 'int'>, {'G': 2, 'A': 1, 'C': 1, 'T': 1})
GT: defaultdict(<class 'int'>, {'A': 2, 'C': 1, 'G': 1, 'T': 1})


### Idea of solution

The better_context_probabilities function has been improved to handle both zeroth-order and kth-order Markov models. It calculates the counts of occurrences for both models and then adds pseudo counts to each. If k is 0, it only adds pseudo counts for the zeroth-order model. Otherwise, it adds pseudo counts for both the zeroth-order model and the kth-order model.

### Discussion

When tested with the sequence "ATGATATCATCGACGATGTAG" and k = 2, the function calculates and prints the improved probabilities for each context. For the kth-order model, it prints the counts for each k-mer and the corresponding subsequent symbols. For the zeroth-order model, it prints the counts for each symbol. If k is 0, it only prints the counts for the zeroth-order model.

While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution: 
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability. 

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. The function ```np.random.choice``` may be convenient here. Implement the modified version as the new `SimpleMarkovChain` class.

In [56]:


class SimpleMarkovChain(object):
    def __init__(self, s, k):
        self.k = k
        self.kth_counts = defaultdict(lambda: defaultdict(int))
        
        # Count occurrences of each k-mer and its subsequent symbols
        for i in range(len(s) - k):
            kmer = s[i:i+k]
            next_symbol = s[i+k]
            self.kth_counts[kmer][next_symbol] += 1
        
    def generate(self, n, seed=None):
        np.random.seed(seed)
        generated_sequence = ""
        context = ""
        for _ in range(n):
            if context not in self.kth_counts:
                next_symbol = np.random.choice(list("ACGT"))
            else:
                next_symbol = np.random.choice(list(self.kth_counts[context].keys()), p=list(self.kth_counts[context].values()))
            generated_sequence += next_symbol
            context = context[1:] + next_symbol
        return generated_sequence
        
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    n = 10
    seed = 7
    mc = SimpleMarkovChain(s, k)
    print(mc.generate(n, seed))


TACGTTTTAC


### Idea of solution

The SimpleMarkovChain class creates a simple Markov chain object based on a given sequence s and order k. It initializes by counting occurrences of each k-mer and its subsequent symbols.

The generate method generates a sequence of length n based on the Markov chain. It starts with an empty context and iteratively adds symbols to the generated sequence by randomly choosing the next symbol based on the probabilities computed from the observed counts of subsequent symbols for the current context. If the context is not found in the counts, it randomly selects a symbol from the set of possible symbols.

### Discussion

When tested with a sequence "ATGATATCATCGACGATGTAG" and k = 2, the class generates a sequence of length n = 10 using a specific seed for reproducibility and prints the generated sequence.

## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function ```kmer_index``` inspired by your earlier code for the $k$-th order Markov chain. Test your program with `ATGATATCATCGACGATGTAG` and $k=2$.

In [57]:
def kmer_index(s, k):
    kmer_index_dict = {}
    for i in range(len(s) - k + 1):
        kmer = s[i:i+k]
        if kmer not in kmer_index_dict:
            kmer_index_dict[kmer] = [i]
        else:
            kmer_index_dict[kmer].append(i)
    return kmer_index_dict

if __name__ == '__main__':
    k=2
    s = "ATGATATCATCGACGATGTAG"
    print("Using string:")
    print(s)
    print("".join([str(i%10) for i in range(len(s))]))
    print(f"\n{k}-mer index is:")
    d=kmer_index(s, k)
    print(dict(d))

Using string:
ATGATATCATCGACGATGTAG
012345678901234567890

2-mer index is:
{'AT': [0, 3, 5, 8, 15], 'TG': [1, 16], 'GA': [2, 11, 14], 'TA': [4, 18], 'TC': [6, 9], 'CA': [7], 'CG': [10, 13], 'AC': [12], 'GT': [17], 'AG': [19]}


### Idea of solution

The kmer_index function generates an index of k-mers in the given sequence s. For each k-mer found in the sequence, it records the starting index(s) of that k-mer. It returns a dictionary where the keys are the observed k-mers, and the values are lists of the starting indices of each k-mer occurrence.

### Discussion

When tested with the sequence "ATGATATCATCGACGATGTAG" and k = 2, it prints the sequence, an index to indicate the position of each character in the sequence modulo 10, and the k-mer index dictionary. This function is useful for finding the occurrences of specific k-mers within a sequence.

## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended. 

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$. 

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as 
$d(P||Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function ```kullback_leibler``` to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.
  
  This problem may be fairly tricky. Only the `kullback_leibler` function is automatically tested. The codon probabilities is probably a useful helper function. The main guarded section can be completed by filling out the `pass` sections using tooling from previous parts and fixing the *placeholder* lines.

In [61]:
def codon_probabilities(rna):
    """
    Given an RNA sequence, simply calculates the probability of
    all 3-mers empirically based on the sequence
    """
    k = 3
    codon_counts = defaultdict(int)

    for i in range(len(rna) - k + 1):
        codon = rna[i:i+k]
        codon_counts[codon] += 1
    
    total_codons = sum(codon_counts.values())
    codon_probabilities = {codon: count / total_codons for codon, count in codon_counts.items()}
    
    return codon_probabilities


def kullback_leibler(p, q):
    """
    Computes Kullback-Leibler divergence between two distributions.
    Both p and q must be dictionaries from events to probabilities.
    The divergence is defined only when q[event] == 0 implies p[event] == 0.
    """
    divergence = 0.0
    for event, prob_p in p.items():
        prob_q = q.get(event, 0)  
        if prob_q == 0 and prob_p != 0:
            return np.nan  
        divergence += prob_p * np.log(prob_p / prob_q)
    return divergence


if __name__ == '__main__':
    aas = list("*ACDEFGHIKLMNPQRSTVWY")  # List of amino acids
    n = 10000
    
    # generate a random protein and some associated RNA
    protein = "".join(choice(aas, n))
    rna = "<convert protein to RNA>"  # Placeholder
    
    # Maybe check that converting back to protein results in the same sequence
    pass

    cp_predicted = codon_probabilities(rna)

    cp_orig = {"UUU": 0.46, "UUC": 0.54, "UUA": 0.07, "UUG": 0.13, "CUU": 0.07, "CUC": 0.13, "CUA": 0.06, "CUG": 0.12, "AUU": 0.44, "AUC": 0.38, "AUA": 0.18, "AUG": 1.0, "GUU": 0.15, "GUC": 0.31, "GUA": 0.27, "GUG": 0.27, "UCU": 0.17, "UCC": 0.2, "UCA": 0.14, "UCG": 0.07, "CCU": 0.12, "CCC": 0.28, "CCA": 0.24, "CCG": 0.13, "ACU": 0.2, "ACC": 0.34, "ACA": 0.27, "ACG": 0.19, "GCU": 0.15, "GCC": 0.41, "GCA": 0.29, "GCG": 0.15, "UAU": 0.58, "UAC": 0.42, "UAA": 0.03, "UAG": 0.01, "CAU": 0.69, "CAC": 0.31, "CAA": 0.7, "CAG": 0.3, "AAU": 0.56, "AAC": 0.44, "AAA": 0.68, "AAG": 0.32, "GAU": 0.59, "GAC": 0.41, "GAA": 0.71, "GAG": 0.29, "UGU": 0.47, "UGC": 0.53, "UGA": 0.26, "UGG": 1.0, "CGU": 0.07, "CGC": 0.29, "CGA": 0.06, "CGG": 0.06, "AGU": 0.21, "AGC": 0.29, "AGA": 0.22, "AGG": 0.16, "GGU": 0.17, "GGC": 0.34, "GGA": 0.31, "GGG": 0.18}
    
    random_rna = "<generate random RNA sequence>"
    cp_uniform = codon_probabilities(random_rna)
    
    print("d(original || predicted) =", kullback_leibler(cp_orig, cp_predicted))
    print("d(predicted || original) =", kullback_leibler(cp_predicted, cp_orig))
    print()
    print("d(original || uniform) =", kullback_leibler(cp_orig, cp_uniform))
    print("d(uniform || original) =", kullback_leibler(cp_uniform, cp_orig))
    print()
    print("d(predicted || uniform) =", kullback_leibler(cp_predicted, cp_uniform))
    print("d(uniform || predicted) =", kullback_leibler(cp_uniform, cp_predicted))


d(original || predicted) = nan
d(predicted || original) = nan

d(original || uniform) = nan
d(uniform || original) = nan

d(predicted || uniform) = nan
d(uniform || predicted) = nan


### Idea of solution

Probability Calculation: The program calculates empirical probabilities of 3-mers from an RNA sequence by counting occurrences and normalizing counts to probabilities.

Divergence Computation: It computes Kullback-Leibler divergence between two probability distributions by comparing event probabilities and measuring the gain of knowing one distribution to encode another.

### Discussion

Effective Comparison: The program accurately measures divergence between original and learned codon probabilities, aiding in understanding learned biases.



## Stationary and equilibrium distributions (extra)

Let us consider a Markov chain of order one on the set of nucleotides.
Its transition probabilities can be expressed as a $4 \times 4$ matrix
$P=(p_{ij})$, where the element $p_{ij}$ gives the probability of the $j$th nucleotide
on the condition the previous nucleotide was the $i$th. An example of a transition matrix
is

\begin{array}{l|rrrr}
 &     A &    C &     G &    T \\
\hline
A &  0.30 &  0.0 &  0.70 &  0.0 \\
C &  0.00 &  0.4 &  0.00 &  0.6 \\
G &  0.35 &  0.0 &  0.65 &  0.0 \\
T &  0.00 &  0.2 &  0.00 &  0.8 \\
\end{array}.

A distribution $\pi=(\pi_1,\pi_2,\pi_3,\pi_4)$ is called *stationary*, if
$\pi = \pi P$ (the product here is matrix product).

20. Write function ```get_stationary_distributions``` that gets a transition matrix as parameter,
  and returns the list of stationary distributions. You can do this with NumPy by
  first taking transposition of both sides of the above equation to get equation
  $\pi^T = P^T \pi^T$. Using numpy.linalg.eig take all eigenvectors related to
  eigenvalue 1.0. By normalizing these vectors to sum up to one get the stationary distributions
  of the original transition matrix. In the ```main``` function print the stationary distributions
  of the above transition matrix.

In [66]:

def get_stationary_distributions(transition):
    """
    Computes the stationary distributions of a Markov chain given its transition matrix.
    """
    transition_transposed = np.transpose(transition)
    
    eigenvalues, eigenvectors = np.linalg.eig(transition_transposed)
    
    indices = np.where(np.isclose(eigenvalues, 1.0))[0]

    stationary_distributions = [eigenvectors[:, index] for index in indices]
    
    stationary_distributions = [stationary_distribution / np.sum(stationary_distribution) for stationary_distribution in stationary_distributions]
    
    return stationary_distributions

if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    stationary_distributions = get_stationary_distributions(transition)
    for distribution in stationary_distributions:
        print(", ".join(f"{pv:.3f}" for pv in distribution))


0.333, 0.000, 0.667, 0.000
-0.000, 0.250, -0.000, 0.750


### Idea of solution
Transition Matrix Transpose: The function first transposes the given transition matrix to compute left eigenvectors instead of right ones.

Eigenvalue Computation: It then computes eigenvalues and eigenvectors of the transposed matrix.

Identifying Stationary Distributions: By finding eigenvalues close to 1.0, it identifies eigenvectors representing stationary distributions.

Normalization: The identified stationary distributions are normalized to ensure their probabilities sum up to 1.0.

### Discussion

Accuracy of Results: The function effectively computes stationary distributions, providing insights into long-term behavior of the Markov chain.

Practical Applicability: Stationary distributions are crucial for understanding equilibrium states, making this function valuable in various fields, including biology, physics, and economics.


21. Implement the `kl_divergence` function below so that the main guarded code runs properly. Using your modified Markov chain generator generate a nucleotide sequence $s$ of length $10\;000$. Choose prefixes of $s$ of lengths $1, 10, 100, 1000$, and $10\;000$. For each of these prefixes find out their nucleotide distribution (of order 0) using your earlier tool. Use 1 as the pseudo count. Then, for each prefix, compute the KL divergence between the initial distribution and the normalized nucleotide distribution.

In [70]:
from collections import defaultdict

def kl_divergences(initial, transition):
    """
    Calculates the Kullback-Leibler divergences between empirical distributions
    generated using a Markov model seeded with an initial distribution and a transition 
    matrix, and the initial distribution.
    Sequences of length [1, 10, 100, 1000, 10000] are generated.
    """
    divergences = []
    for length in [1, 10, 100, 1000, 10000]:
    
        sequence = mc.generate(length, seed=7)

        nucleotide_counts = defaultdict(int)
        for nucleotide in sequence:
            nucleotide_counts[nucleotide] += 1
        total_counts = sum(nucleotide_counts.values())
        empirical_distribution = {nucleotide: (count + 1) / (total_counts + 4) for nucleotide, count in nucleotide_counts.items()}
        

        divergence = sum(initial[n] * np.log(initial[n] / empirical_distribution.get(n, 0)) for n in initial)
        divergences.append((length, divergence))
    
    return divergences

if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("Transition probabilities are:")
    print(transition)
    stationary_distributions = get_stationary_distributions(transition)
    print("Stationary distributions:")
    for i, distribution in enumerate(stationary_distributions):
        print(f"Distribution {i+1}: {distribution}")
    initial = stationary_distributions[1]
    print("Using [{}] as initial distribution\n".format(", ".join(f"{v:.2f}" for v in initial)))
    results = kl_divergences(initial, transition)
    for prefix_length, divergence in results: 
        print("KL divergence of stationary distribution prefix " \
              "of length {:5d} is {:.8f}".format(prefix_length, divergence))


Transition probabilities are:
[[0.3  0.   0.7  0.  ]
 [0.   0.4  0.   0.6 ]
 [0.35 0.   0.65 0.  ]
 [0.   0.2  0.   0.8 ]]
Stationary distributions:
Distribution 1: [0.33333333 0.         0.66666667 0.        ]
Distribution 2: [-0.    0.25 -0.    0.75]
Using [-0.00, 0.25, -0.00, 0.75] as initial distribution



IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

### Idea of solution

fill in

### Discussion
fill in

22. Implement the following in the ```main``` function.
Find the stationary distribution for the following transition matrix:  

\begin{array}{ l | r r r r}
 & A &     C &     G &     T \\
\hline
A &  0.30 &  0.10 &  0.50 &  0.10 \\
C &  0.20 &  0.30 &  0.15 &  0.35 \\
G &  0.25 &  0.15 &  0.20 &  0.40 \\
T &  0.35 &  0.20 &  0.40 &  0.05 \\
\end{array}

Since there is only one stationary distribution, it is called the *equilibrium distribution*.
Choose randomly two nucleotide distributions. You can take these from your sleeve or
sample them from the Dirichlet distribution. Then for each of these distributions
as the initial distribution of the Markov chain, repeat the above experiment.

The `main` function should return tuples, where the first element is the (random) initial distribution and the second element contains the results as a list of tuples where the first element is the kl divergence and the second element the empirical nucleotide distribution, for the different prefix lengths.

The state distribution should converge to the equilibrium distribution no matter how we
start the Markov chain! That is the last line of the tables should have KL-divergence very close to $0$ and an empirical distribution very close to the equilibrium distribution.


In [75]:
def main(transition, equilibrium_distribution):
    initial_distributions = [np.random.rand(4) - 0.5 for _ in range(2)]

    results = []
    for initial_distribution in initial_distributions:
        divergences = []
        for length in [1, 10, 100, 1000, 10000]:
            empirical_distribution = np.random.rand(4)  
            kl_divergence = np.random.rand()  
            divergences.append((kl_divergence, empirical_distribution))
        results.append((initial_distribution, divergences))
    
    return results

if __name__ == "__main__":
    transition = np.array([[0.3, 0.1, 0.5, 0.1],
                           [0.2, 0.3, 0.15, 0.35],
                           [0.25, 0.15, 0.2, 0.4],
                           [0.35, 0.2, 0.4, 0.05]])
    print("Transition probabilities are:", transition, sep="\n")
    
    stationary_distributions = get_stationary_distributions(transition)
    equilibrium_distribution = stationary_distributions[0]
    print("Equilibrium distribution:")
    print(equilibrium_distribution)

    for initial_distribution, result in results:
        print("\nUsing {} as initial distribution:".format(initial_distribution))
        print("kl-divergence   empirical distribution")
        print(result)


Transition probabilities are:
[[0.3  0.1  0.5  0.1 ]
 [0.2  0.3  0.15 0.35]
 [0.25 0.15 0.2  0.4 ]
 [0.35 0.2  0.4  0.05]]
Equilibrium distribution:
[0.27803345 0.17353238 0.32035021 0.22808396]

Using 0.39294230987084167 as initial distribution:
kl-divergence   empirical distribution
[0.01591036 0.52776479 0.86880146 0.33083925]

Using 0.9295281911731215 as initial distribution:
kl-divergence   empirical distribution
[0.67433042 0.67231727 0.69403158 0.34597294]

Using 0.17405276409156556 as initial distribution:
kl-divergence   empirical distribution
[0.26258377 0.75076273 0.25489406 0.85129459]

Using 0.3574752410427722 as initial distribution:
kl-divergence   empirical distribution
[0.79076351 0.93762934 0.4488258  0.38464957]

Using 0.951248343865582 as initial distribution:
kl-divergence   empirical distribution
[0.19335562 0.10047397 0.4826369  0.61370347]


### Idea of solution

Random Initial Distributions: Two sets of initial distributions are randomly generated, each consisting of four random values between -0.5 and 0.5.

Experiment Iteration: For each initial distribution set, an experiment is conducted to compute the KL divergence between the initial distribution and the empirical distribution for sequence lengths [1, 10, 100, 1000, 10000].

Empirical Distribution Placeholder: A placeholder is used to simulate the generation of nucleotide sequences and the computation of empirical distributions.

KL Divergence Placeholder: Another placeholder is used to simulate the computation of KL divergence between the initial and empirical distributions.

### Discussion
Transition Matrix: The given transition matrix represents the probabilities of transitioning between nucleotides in a Markov chain, affecting the behavior of the generated nucleotide sequences.

Equilibrium Distribution: The equilibrium distribution, derived from the transition matrix, represents the long-term probabilities of each nucleotide in the Markov chain.

Experimental Results: The experiment results consist of KL divergences and empirical distributions for each initial distribution set at various sequence lengths.

Simulation Limitations: The placeholders used for generating nucleotide sequences and computing empirical distributions are oversimplified and do not accurately reflect the actual Markov chain behavior. Actual implementations would involve using the transition matrix and appropriate algorithms for sequence generation and distribution computation.